In [201]:
import re
import urllib.request
import json
from pymystem3 import Mystem
m = Mystem()

In [202]:
def if_url_works(word):
    codes = ['a 42 520', 'b 1 654', 'c 2 471', 'd 19 571', 'e 1 554', 'f 11 50', 'h 11 103', 'i 1 236',
             'j 14 298', 'l 13 992', '3 1 245', 'm 5 383', 'n 3 709', 'o 1 299', 'p 2 488', 'q 186 999',
             '4 5 554', 'r 2 511', 's 13 990', '2 5 157', 't 6 686', 'u 6 212', 'v 3 388', 'x 31 211',
             'w 13 100', 'y 8 188', 'z 8 333', '1 11 31', '5 37 256', '6 4 19', '7 1 76']
    log = ''
    
    
    for code in codes:
        l = code.split()[0]
        n1 = int(code.split()[1])
        n2 = int(code.split()[2])
        while n1 <= n2:
            try:
                #time.sleep(2)
                page = urllib.request.urlopen('http://old.kpfu.ru/infres/slovar1/gn.php?' + word + '&' + l + str(n1) + '.gn&1#1')
                page_read = page.read().decode('cp1251')
                if 'Failed opening file' in page_read:
                    log += 'nf:' + word + '&' + l + str(n1) + '\n'
                else:
                    log += 'good:' + word + '&' + l + str(n1) + '\n'
                    print('http://old.kpfu.ru/infres/slovar1/gn.php?' + word + '&' + l + str(n1) + '.gn&1#1')
                n1 += 1
            except Exception:
                log += 'nsfw:' + word + '&' + l + str(n1) + '\n'
                n1 += 1
                
                
    return log

In [203]:
def word2url(word):
    new_word = word
    num_let = ['а %E0', 'б %E1', 'в %E2', 'г %E3', 'д %E4', 'е %E5', 'ж %E6', 'з %E7', 'и %E8', 'й %E9', 'к %EA', 'л %EB',
               'м %EC', 'н %ED', 'о %EE', 'п %EF', 'р %F0', 'с %F1', 'т %F2', 'у %F3', 'ф %F4', 'х %F5', 'ц %F6', 'ч %F7',
               'ш %F8', 'щ %F8', 'ь %FC', 'ы %FB', 'э %FD', 'ю %FE', 'я %FF', 'IV @IV', 'I @I', '@I@I@I @III', '@I@I @II']
    for two in num_let:
        new_word = new_word.replace(two.split()[0], two.split()[-1])
    new_word = new_word.replace('\n', '')
    new_word = new_word.replace(' ', '')
    return new_word

In [204]:
def find_urls():
    words = []

    with open('words.txt', 'r', encoding='utf-8') as f:
        for line in f:
            words.append(line)
    return words

In [205]:
def root2rt(root):
    rt = root.replace('\n', '')
    rt = rt.replace('j', '')
    rt = rt.replace('I', '').replace('1', '').replace('2', '').replace('3', '')
    rt = rt.replace('<i>', '').replace('</i>', '')
    rt = rt.replace('(', '').replace(')', '')
    if ' ' in rt:
        rt = re.split(' ', rt)[0]
    elif '/' in rt:
        rt = re.split('/', rt)[-1]
    return rt

In [206]:
def read_link(link, idshnik):
    page = urllib.request.urlopen(link)
    page_read = page.read().decode('cp1251')
    depth = 0
    nods = []
    last_words = []
    for line in page_read.splitlines():
        if 'Гнездо для слова' in line:
            depth = 0
            
            re_word = '(<FONT COLOR=#FF0000><b>)(.*?)(</b></FONT>)'
            word = re.findall(re_word, line)[0][1].replace('\'', '')
            if '</FONT>(' in line:
                re_end = '\(.*?\)'
                ending = re.findall(re_end, line)[-1].replace('(', '').replace(')', '')
                word += ending
            word = word.replace('<i>', '').replace('</i>', '')
            word = word.replace('(и', '(ия)')
            word = word.replace('\'', '')
            if word not in '     /n/t':
                nod = {'root': word, 'mettype': 'else', 'met': 'none',
                       'ambiguous': [], 'non-ambiguous': [], 'depth': 0,
                       'id': idshnik, 'PoS': m.analyze(root2rt(word))[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]}
                idshnik += 1
                last_words.append(word)
                nods.append(nod)


        elif re.search('[А-Яа-яЁё]', line):
            for each in re.findall('<ul>', line):
                depth += 1
            for each in re.findall('</ul>', line):
                depth -= 1

            re_word2 = '[-\'()А-Яа-яЁёIV12345\[j]+' 
            word2 = ''

            for part in re.findall(re_word2, line):
                if part == 'См' or part == 'сущ':
                    break
                elif 'нескл' in part or 'ср' in part or 'прич' in part or 'прил' in part or 'нареч' in part:
                    break
                elif 'I' in part or 'V' in part or '1' in part or '2' in part or '3' in part or '4' in part or '5' in part:
                    word2 += ' ' + part
                    break
                elif '\[' in part:
                    word2 = ''.join(part[1:-1])
                else:
                    word2 += part

            word2 = word2.replace('-', '')
            word2 = word2.replace(')', '')
            word2 = word2.replace('(', '')
            word2 = word2.replace('\'', '')
            word2 = word2.replace('0', '')
            word2 = word2.replace('[', '/')
            
            word2 = word2.replace('поjоный', 'поеный')
            word2 = word2.replace('солоный', 'соленый')
            word2 = word2.replace('жы', 'жи')
            word2 = word2.replace('шы', 'ши')

            t = True
            for n in nods:
                if n['root'] == ['word2']:
                    nod2 = n
                    print(n['root'])
                    t = False
                    
            mettype2 = 'else'
            met2 = 'none'

            
            if word2 not in '     /n/t':
                if t and 'любый' not in word2:
                    nod2 = {'root': word2, 'mettype': mettype2, 'met': met2,
                            'ambiguous': [], 'non-ambiguous': [], 'depth': 0,
                            'id': idshnik, 'PoS': m.analyze(root2rt(word2))[0]['analysis'][0]['gr'].split('=')[0].split(',')[0]}
                    idshnik += 1
                elif t:
                    nod2 = {'root': word2, 'mettype': mettype2, 'met': met2,
                            'ambiguous': [], 'non-ambiguous': [], 'depth': 0,
                            'id': idshnik, 'PoS': 'A'}


                if '<FONT COLOR=#FF0000><b>' not in line:
                    mettype2 = 'else'
                    met2 = 'none'

                else:
                    met2 = re.search('<FONT COLOR=#FF0000><b>(.*?)<', line).group(1)
                    if word2.endswith(met2):
                        mettype2 = 'suf'
                    elif word2.startswith(met2):
                        mettype2 = 'pref'
                    else:
                        mettype2 = 'else'
                    for e in endings:
                        if word2.endswith(met2+e):
                            mettype2 = 'suf'

                if nod2['depth'] == 0:
                    nod2['mettype'] = mettype2
                    nod2['met'] = met2
                    nod2['depth'] = depth
                    nod2['id'] = idshnik
                    idshnik += 1

                nods.append(nod2)

                if len(last_words) <= depth:
                    last_words.append(word2)
                elif depth < 0:
                    print(word2, line)
                else:
                    last_words[depth] = word2

                if ',,1' in line or ',,2' in line:
                    for n in nods:
                        if depth > 0:
                            if n['root'] == last_words[depth-1]:
                                n['ambiguous'].append(word2)         
                else:
                    for n in nods:
                        if depth > 0:
                            if n['root'] == last_words[depth-1]:
                                n['non-ambiguous'].append(word2)
                            
    return(nods, idshnik)

In [207]:
idshnik = 0
endings = ['ий', 'ый', 'ой', 'ая', 'ое', 'ие', 'а', 'я', 'ы', 'и', 'е', 'у', 'ю', 'ть', 'ться', 'ся']
links = []


with open('links.txt', 'r', encoding='utf-8') as f:
    for line in f:
        links.append(line)
            
print(read_link('http://old.kpfu.ru/infres/slovar1/gn.php?%E0%E2%E3%F3%F1%F2&b227.gn&1#1', idshnik))

([{'root': 'береза', 'mettype': 'else', 'met': 'none', 'ambiguous': [], 'non-ambiguous': ['березка', 'березонька', 'березина', 'березник', 'березняк', 'березовик', 'березовый', 'подберезовик', 'подберезник'], 'depth': 0, 'id': 0, 'PoS': 'S'}, {'root': 'березка', 'mettype': 'suf', 'met': 'к', 'ambiguous': [], 'non-ambiguous': [], 'depth': 1, 'id': 2, 'PoS': 'S'}, {'root': 'березонька', 'mettype': 'suf', 'met': 'оньк', 'ambiguous': [], 'non-ambiguous': [], 'depth': 1, 'id': 4, 'PoS': 'S'}, {'root': 'березина', 'mettype': 'suf', 'met': 'ин', 'ambiguous': [], 'non-ambiguous': [], 'depth': 1, 'id': 6, 'PoS': 'S'}, {'root': 'березник', 'mettype': 'suf', 'met': 'ник', 'ambiguous': [], 'non-ambiguous': [], 'depth': 1, 'id': 8, 'PoS': 'S'}, {'root': 'березняк', 'mettype': 'else', 'met': "ня'к", 'ambiguous': [], 'non-ambiguous': [], 'depth': 1, 'id': 10, 'PoS': 'S'}, {'root': 'березовик', 'mettype': 'suf', 'met': 'овик', 'ambiguous': [], 'non-ambiguous': [], 'depth': 1, 'id': 12, 'PoS': 'S'}, {'

In [208]:
idshnik = 0
json_nests = []
for l in links:    
    nest, idshnik = read_link(l, idshnik)
    if nest[0]['root'] not in json_nests:
        json_nests.append({nest[0]['root']: nest})

In [200]:
file_object = open('nests.json', 'w', encoding='utf-8')
json.dump(json_nests, file_object, ensure_ascii=False)
file_object.close()

In [209]:
print(len(json_nests))
print(idshnik)

1565
148998


In [210]:
def nprint(nest, root):
    trees ={}
    nr = 0
    for n in nest.values():
        nroot = []
        for l in n:
            if (
                root == l['root'] or
                l['root'].startswith(root + ' ') or
                l['root'].endswith('/' + root)
                ):
                trees[l['root']] = [l['root']]
                
        for r in trees:
            ts = [r]
            for l in n[::-1]:
                for t in ts:
                    if t in l['non-ambiguous'] or t in l['ambiguous']:
                        ts.append(l['root'])
            trees[r] = ts
        
        for r in trees:
            ts = []
            for l in n:
                if l['root'] in ts or l['root'] == r: 
                    for na in l['non-ambiguous']:
                        if na not in ts:
                            ts.append(na)
                    for a in l['ambiguous']:
                        if a not in ts:
                            ts.append(a)
            trees[r].extend(ts)
            
        for r in trees:
            for l in n:
                if l['root'] in trees[r]:
                    string = ''
                    for d in range(l['depth']):
                        string += '\t'
                    string += l['root']
                    print(string)

In [215]:
nprint(json_nests[163], 'вынуть')

вынуть
	вынуться
		выниматься 1
	вынимать
		выниматься 2
		выниманиjе/вынимание
		повынимать
	выем
		выемка
			выемочный
			выемчатый 1
		выемщик
		выемный
		выемной
		выемный
		выемчатый 2
	повынуть


In [214]:
print(json_nests[163])

{'вынуть': [{'root': 'вынуть', 'mettype': 'else', 'met': 'none', 'ambiguous': [], 'non-ambiguous': ['вынуться', 'вынимать', 'выем', 'повынуть'], 'depth': 0, 'id': 16407, 'PoS': 'V'}, {'root': 'вынуться', 'mettype': 'suf', 'met': 'ся', 'ambiguous': ['выниматься 1'], 'non-ambiguous': [], 'depth': 1, 'id': 16409, 'PoS': 'V'}, {'root': 'выниматься 1', 'mettype': 'else', 'met': "а'", 'ambiguous': [], 'non-ambiguous': [], 'depth': 2, 'id': 16411, 'PoS': 'V'}, {'root': 'вынимать', 'mettype': 'else', 'met': "а'", 'ambiguous': ['выниматься 2'], 'non-ambiguous': ['выниманиjе/вынимание', 'повынимать'], 'depth': 1, 'id': 16413, 'PoS': 'V'}, {'root': 'выниматься 2', 'mettype': 'else', 'met': 'ся', 'ambiguous': [], 'non-ambiguous': [], 'depth': 2, 'id': 16415, 'PoS': 'V'}, {'root': 'выниманиjе/вынимание', 'mettype': 'else', 'met': 'ниj', 'ambiguous': [], 'non-ambiguous': [], 'depth': 2, 'id': 16417, 'PoS': 'S'}, {'root': 'повынимать', 'mettype': 'pref', 'met': 'по', 'ambiguous': [], 'non-ambiguous':